# Using TensorBoard in Workbenches

Run and open TensorBoard to monitor the training process.

### 1. Import the required libraries and packages.

You can safely ignore the TensorFlow import warnings.
TensorFlow typically produces these warnings on CPU-only environments where the use of accelerators, such as GPUs or certain CPU instruction sets, is limited or not available.

In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 69.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 155.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 270.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 115.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 178.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 148.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 185.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 196.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 173.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2


In [14]:
import os
import datetime
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

### 2. Load and Preprocess the Data

Split the data into train, test, and validation subsets.

In [15]:
data = pd.read_csv('./data/diabetes.csv')

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0
)

X_test, X_validation, y_test, y_validation = train_test_split(
    X_test,
    y_test,
    test_size=0.3,
    random_state=0
)

print(f"Number of samples in training set: {X_train.shape[0]}")
print(f"Number of samples in test set: {X_test.shape[0]}")

Number of samples in training set: 537
Number of samples in test set: 161


### 4. Create and train the model.

Define the model.
Use `accuracy` as the metric to evaluate the model.

In [7]:
# Seed for reproducible results
tf.random.set_seed(10)
tf.keras.utils.set_random_seed(10)

model = tf.keras.Sequential([
    tf.keras.layers.Input((8,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Define the logging callback.

In [8]:
log_dir = "logs/training/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

Train the model.
Use `validation_data` to validate the model after each epoch.

TensorFlow reports the value of the loss function and the `accuracy` metric for both the training and the validation subsets.

In [9]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_validation, y_validation),
    callbacks=[logging_callback]
)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4584 - loss: 22.0053 - val_accuracy: 0.5286 - val_loss: 3.6104
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4407 - loss: 3.0189 - val_accuracy: 0.5429 - val_loss: 2.1024
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4790 - loss: 1.6276 - val_accuracy: 0.4857 - val_loss: 1.4470
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5679 - loss: 1.3680 - val_accuracy: 0.5429 - val_loss: 1.1018
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5883 - loss: 1.1866 - val_accuracy: 0.5286 - val_loss: 1.0239
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6242 - loss: 1.0676 - val_accuracy: 0.5571 - val_loss: 0.9808
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6341 - loss: 0.9677 - val_accuracy: 0.6000 - val_loss: 0.8690
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6393 - loss: 0.8933 - val_accuracy: 0.6000 -

### 5. Run TensorBoard

Load the `tensorboard` notebook extension.

In [10]:
%load_ext tensorboard

Set the `TENSORBOARD_PROXY_URL` environment variable to access TensorBoard from this notebook.

In [11]:
os.environ["TENSORBOARD_PROXY_URL"] = os.getenv("NB_PREFIX") + "/proxy/6006/"

print("TensorBoard URL:", os.environ["TENSORBOARD_PROXY_URL"])

TensorBoard URL: /notebook/fpgpocnotebooks/fpgwbdsdataconnection/proxy/6006/


Run the TensorBoard server.

TensorBoard plots accuracy and loss for each epoch, for both the training and validation subsets.

Note that, as the training progresses, the loss value decreases and the accuracy of the model increases.

The fact that the validation subset accuracy remains close to the training accuracy means that the model is generalizing well on samples not seen during training.


In [16]:
%tensorboard --logdir logs/training

Reusing TensorBoard on port 6006 (pid 11949), started 14:10:42 ago. (Use '!kill 11949' to kill it.)